<div style='background-color: orange'>
<a id="TableOfContents"></a>
    <h1 style='text-align: center'>
        <b><i>
            TABLE OF CONTENTS:
        </i></b></h1>
<li><a href='#imports'>Imports</a></li>
<li><a href="#acquire">Acquire</a></li>
<li><a href='#prepare'>Prepare</a></li>
<li><a href="#wrangle">Wrangle</a></li>
<li><a href='#misc'>Miscellaneous</a></li>

<div style='background-color: orange'>
<a id="imports"></a>
    <h1 style='text-align: center'>
        <b><i>
            Imports
        </i></b></h1>
<li><a href='#TableOfContents'>Table of Contents</a></li>

In [3]:
# Vectorization and tables
import numpy as np
import pandas as pd

# .py files
import wrangle as w

<div style='background-color: orange'>
<a id="acquire"></a>
    <h1 style='text-align: center'>
        <b><i>
            Acquire
        </i></b></h1>
<li><a href='#TableOfContents'>Table of Contents</a></li>

Acquire everything from the vanilla mass_shooters database via excel sheet

- mass_shooters Vanilla Shape:
    - Rows: 189
    - Columns: 153

In [4]:
# Acquire the vanilla data
mass_shooters = pd.read_excel('mass_shooters.xlsx', sheet_name='Full Database', header=1)
mass_shooters.shape

(189, 153)

In [5]:
# Verify .py file functionality
mass_shooters_py = w.acquire_mass_shooters()
mass_shooters_py.shape

(189, 153)

<div style='background-color: orange'>
<a id="prepare"></a>
    <h1 style='text-align: center'>
        <b><i>
            Prepare
        </i></b></h1>
<li><a href='#TableOfContents'>Table of Contents</a></li>
<li><a href='#preparedrop'>Drop Data</a></li>
<li><a href='#preparenull'>Null Handling</a></li>
<li><a href='#preparedtypes'>Dtype Cleaning</a></li>
<li><a href='#preparetext'>Text Modifications</a></li>
<li><a href='#preparesummary'>Summary of Preparation</a></li>

<a id='preparedrop'></a>
<h3><b><i>
    Drop Data
</i></b></h3>
<li><a href='#prepare'>Prepare Top</a></li>

- 1 Row
    - Missing majority of info
- 47 columns
    - Useless in scope of predictive value
    - Percent nulls above 20%

In [6]:
# Remove row '145, 146' (Too many nulls)
mass_shooters = mass_shooters.drop(mass_shooters[mass_shooters['Case #'] == '145, 146'].index)

In [7]:
# Drop cols that aren't necessary
# Generally anything that would've been known during
# and/or after the shooting
# 29 columns
drop_perpetratorname_cols = [
    'Shooter Last Name',
    'Shooter First Name'
]

drop_date_cols = [
    'Full Date'
]

drop_location_cols = [
    'Street Number',
    'Street Name',
    'Zip Code',
    'Latitude',
    'Longitude',
    'State Code',
    'Region',
    'Metro/Micro Statistical Area Type',
    'Location',
    'Insider or Outsider',
    'Workplace Shooting',
    'Multiple Locations',
    'Other Location',
    'Armed Person on Scene',
    'Specify Armed Person'
]

drop_victim_cols = [
    'Family Member Victim',
    'Romantic Partner Victim',
    'Kidnapping or Hostage Situation'
]

drop_weapons_cols = [
    'Total Firearms Brought to the Scene',
    'Other Weapons or Gear',
    'Specify Other Weapons or Gear',
]

drop_resolutionofcase_cols = [
    'On-Scene Outcome',
    'Who Killed Shooter On Scene',
    'Attempt to Flee',
    'Insanity Defense',
    'Criminal Sentence'
]

In [8]:
# Drop the above cols
mass_shooters = mass_shooters.drop(columns=drop_perpetratorname_cols)
mass_shooters = mass_shooters.drop(columns=drop_date_cols)
mass_shooters = mass_shooters.drop(columns=drop_location_cols)
mass_shooters = mass_shooters.drop(columns=drop_victim_cols)
mass_shooters = mass_shooters.drop(columns=drop_weapons_cols)
mass_shooters = mass_shooters.drop(columns=drop_resolutionofcase_cols)

In [9]:
mass_shooters.shape

(188, 124)

In [10]:
# Drop cols based off of a null percent cutoff
# 21 columns
mass_shooters, drop_null_pct_dict = drop_nullpct(mass_shooters, 0.20)

In [11]:
# Get list and percentages of everything dropped
pd.DataFrame(drop_null_pct_dict)

,column_name,percent_null
0,Height,0.702128
1,Weight,0.760638
2,Religion,0.505319
3,Education,0.265957
4,School Performance,0.531915
5,School Performance Specified,0.526596
6,Birth Order,0.420213
7,Number of Siblings,0.255319
8,Older Siblings,0.436170
9,Younger Siblings,0.446809


In [12]:
mass_shooters.shape

(188, 103)

In [13]:
drop_notmuchinsight_cols = [
    'Adult Trauma'
]

---

<a id='preparenull'></a>
<h3><b><i>
    Null Handling
</i></b></h3>
<li><a href='#prepare'>Prepare Top</a></li>

In [14]:
# Identify cols with nulls
has_nulls = check_nulls(mass_shooters)
len(has_nulls)

45

45 columns out of 106 columns with nulls:

- Fill with mode
    - 44 columns
- Fill uniquely
    - 1 columns ('None')

In [15]:
# Filling nulls
mass_shooters['Signs of Crisis Expanded'] = mass_shooters['Signs of Crisis Expanded'].fillna('None')
for col in has_nulls:
    mass_shooters[col] = mass_shooters[col].fillna(mass_shooters[col].mode()[0])

In [16]:
# Recheck for any nulls
has_nulls_verify = check_nulls(mass_shooters)
len(has_nulls_verify)

0

---

<a id='preparedtypes'></a>
<h3><b><i>
    Dtype Cleaning
</i></b></h3>
<li><a href='#prepare'>Prepare Top</a></li>

103 columns out of 103 columns:

- Object
    - 20 Columns
        - 'Case #'
        - 'Day of Week'
        - 'Day'
        - 'City'
        - 'State'
        - 'County'
        - 'Race'
        - 'Employment Status',
        - 'Criminal Record',
        - 'Part I Crimes',
        - 'Part II Crimes',
        - 'Domestic Abuse Specified',
        - 'Adult Trauma',
        - 'Recent or Ongoing Stressor',
        - 'Signs of Crisis Expanded',
        - 'Voluntary or Mandatory Counseling',
        - 'Mental Illness',
        - 'Known Family Mental Health History',
        - 'Substance Use',
        - 'Known Prejudices\xa0'
- Float
    - 78 Columns
        - 'Age',
        - 'Gender',
        - 'Immigrant',
        - 'Sexual Orientation',
        - 'Relationship Status',
        - 'Children',
        - 'Employment Type\xa0',
        - 'Military Service',
        - 'Community Involvement',
        - 'Known to Police or FBI',
        - 'Highest Level of Justice System Involvement',
        - 'History of Physical Altercations',
        - 'History of Animal Abuse',
        - 'History of Domestic Abuse',
        - 'History of Sexual Offenses',
        - 'Gang Affiliation',
        - 'Terror Group Affiliation',
        - 'Known Hate Group or Chat Room Affiliation',
        - 'Violent Video Games',
        - 'Bully',
        - 'Bullied',
        - 'Raised by Single Parent',
        - 'Parental Divorce / Separation',
        - 'Parental Death in Childhood',
        - 'Parental Suicide',
        - 'Childhood Trauma',
        - 'Physically Abused',
        - 'Sexually Abused',
        - 'Emotionally Abused',
        - 'Neglected',
        - 'Mother Violent Treatment',
        - 'Parental Substance Abuse',
        - 'Parent Criminal Record',
        - 'Family Member Incarcerated',
        - 'Signs of Being in Crisis',
        - 'Timeline of Signs of Crisis',
        - 'Inability to Perform Daily Tasks',
        - 'Notably Depressed Mood',
        - 'Unusually Calm or Happy',
        - 'Rapid Mood Swings',
        - 'Increased Agitation',
        - 'Abusive Behavior',
        - 'Isolation',
        - 'Losing Touch with Reality',
        - 'Paranoia',
        - 'Suicidality',
        - 'Prior Hospitalization',
        - 'Voluntary or Involuntary Hospitalization',
        - 'Prior Counseling',
        - 'Psychiatric Medication',
        - 'Treatment 6 Months Prior to Shooting',
        - 'FASD (Fetal Alcohol Spectrum Disorder)',
        - 'Autism Spectrum',
        - 'Health Issues',
        - 'Head Injury / Possible TBI',
        - 'Motive: Racism/Xenophobia',
        - 'Motive: Religious Hate',
        - 'Motive: Misogyny',
        - 'Motive: Homophobia',
        - 'Motive: Employment Issue',
        - 'Motive: Economic Issue',
        - 'Motive: Legal Issue',
        - 'Motive: Relationship Issue',
        - 'Motive: Interpersonal Conflict\xa0',
        - 'Motive: Fame-Seeking',
        - 'Motive: Other\xa0',
        - 'Motive: Unknown',
        - 'Role of Psychosis in the Shooting',
        - 'Social Media Use\xa0',
        - 'Leakage\xa0',
        - 'Interest in Past Mass Violence',
        - 'Relationship with Other Shooting(s)',
        - 'Legacy Token',
        - 'Pop Culture Connection',
        - 'Planning',
        - 'Performance',
        - 'Interest in Firearms',
        - 'Firearm Proficiency'
- Int
    - 5 Columns
        - 'Month', 
        - 'Year', 
        - 'Urban/Suburban/Rural', 
        - 'Number Killed', 
        - 'Number Injured']

In [17]:
# Ensure there are no ' ' values...
for col in mass_shooters.columns:
    if mass_shooters[col].dtype == 'object':
        if mass_shooters[col].apply(lambda x: isinstance(x, str) and x.isspace()).any():
            mass_shooters[col].replace(r'^\s*$', np.nan, regex=True, inplace=True)
            mass_shooters[col].fillna(mass_shooters[col].mode()[0], inplace=True)

In [45]:
# Change Object Columns
mass_shooters['Case #'] = mass_shooters['Case #'].astype(int)
mass_shooters['Day'] = np.where(mass_shooters['Day'] == '19-20', 19, mass_shooters['Day'])
mass_shooters['Day'] = mass_shooters['Day'].astype(int)
mass_shooters['Race'] = np.where(mass_shooters['Race'] == 'Moroccan', 6, mass_shooters['Race'])
mass_shooters['Race'] = np.where(mass_shooters['Race'] == 'Bosnian', 7, mass_shooters['Race'])
mass_shooters['Race'] = mass_shooters['Race'].astype(int)
mass_shooters['Criminal Record'] = np.where(mass_shooters['Criminal Record'] == '1`', 1, mass_shooters['Criminal Record'])
mass_shooters['Criminal Record'] = mass_shooters['Criminal Record'].astype(int)
mass_shooters['Adult Trauma'] = np.where(mass_shooters['Adult Trauma'] == '1, 3', 1, mass_shooters['Adult Trauma'])
mass_shooters['Adult Trauma'] = mass_shooters['Adult Trauma'].astype(int)
mass_shooters['Voluntary or Mandatory Counseling'] = np.where(mass_shooters['Voluntary or Mandatory Counseling'] == '1, 2', 3, mass_shooters['Voluntary or Mandatory Counseling'])
mass_shooters['Voluntary or Mandatory Counseling'] = mass_shooters['Voluntary or Mandatory Counseling'].astype(int)
mass_shooters['Mental Illness'] = np.where(mass_shooters['Mental Illness'] == '1, 2', 5, mass_shooters['Mental Illness'])
mass_shooters['Mental Illness'] = mass_shooters['Mental Illness'].astype(int)
mass_shooters['Known Family Mental Health History'] = np.where(mass_shooters['Known Family Mental Health History'] == '1, 2', 1, mass_shooters['Known Family Mental Health History'])
mass_shooters['Known Family Mental Health History'] = mass_shooters['Known Family Mental Health History'].astype(int)


In [50]:
mass_shooters['Immigrant'].value_counts()

0.0    160
1.0     28
Name: Immigrant, dtype: int64

In [ ]:
# Change Float Columns
mass_shooters['Age'] = mass_shooters['Age'].astype(int)
mass_shooters['Gender'] = np.where(mass_shooters['Gender'] == 3.0, 0, mass_shooters['Gender'])
mass_shooters['Gender'] = mass_shooters['Gender'].astype(int)
mass_shooters['Immigrant'] = mass_shooters['Immigrant'].astype(int)

---

<a id='preparetext'></a>
<h3><b><i>
    Text Modification
</i></b></h3>
<li><a href='#prepare'>Prepare Top</a></li>

---

<a id='preparesummary'></a>
<h3><b><i>
    Summary of Preparation
</i></b></h3>
<li><a href='#prepare'>Prepare Top</a></li>

- Dropped Data
    - 1 Row
        - Missing majority of info
    - 47 columns
        - Useless in scope of predictive value
        - Percent nulls above 20%
- Dtype Cleaning
- Null Handling
- Text Modifications

<div style='background-color: orange'>
<a id="wrangle"></a>
    <h1 style='text-align: center'>
        <b><i>
            Wrangle
        </i></b></h1>
<li><a href='#TableOfContents'>Table of Contents</a></li>

<div style='background-color: orange'>
<a id="misc"></a>
    <h1 style='text-align: center'>
        <b><i>
            Miscellaneous
        </i></b></h1>
<li><a href='#TableOfContents'>Table of Contents</a></li>

In [1]:
def drop_nullpct(df, percent_cutoff):
    '''
    Takes in a dataframe and a percent_cutoff of nulls to drop a column on
    and returns the new dataframe and a dictionary of dropped columns and their pct...
    
    INPUT:
    df = pandas dataframe
    percent_cutoff = Null percent cutoff amount
    
    OUTPUT:
    new_df = pandas dataframe with dropped columns
    drop_null_pct_dict = dict of column names dropped and pcts
    '''
    drop_null_pct_dict = {
        'column_name' : [],
        'percent_null' : []
    }
    for col in df:
        pct = df[col].isna().sum() / df.shape[0]
        if pct > 0.20:
            df = df.drop(columns=col)
            drop_null_pct_dict['column_name'].append(col)
            drop_null_pct_dict['percent_null'].append(pct)
    new_df = df
    return new_df, drop_null_pct_dict

In [2]:
def check_nulls(df):
    '''
    Takes a dataframe and returns a list of columns that has at least one null value
    
    INPUT:
    df = pandas dataframe
    
    OUTPUT:
    has_nulls = List of column names with at least one null
    '''
    has_nulls = []
    for col in df:
        nulls = df[col].isna().sum()
        if nulls > 0:
            has_nulls.append(col)
    return has_nulls